In [31]:
import pandas as pd
from sklearn import metrics 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pickle

In [32]:
df = pd.read_csv('transactions_train.csv')

In [33]:
df[df['isFraud']==0].count()

step              6343476
type              6343476
amount            6343476
nameOrig          6343476
oldbalanceOrig    6343476
newbalanceOrig    6343476
nameDest          6343476
oldbalanceDest    6343476
newbalanceDest    6343476
isFraud           6343476
dtype: int64

In [34]:
df = df.drop(['nameOrig','nameDest'], axis =1)

In [35]:
df = pd.get_dummies(df)
df = df[(df["type_CASH_OUT"]==1) | (df["type_TRANSFER"]==1)]
df = df.drop(['type_CASH_IN','type_CASH_OUT','type_DEBIT','type_PAYMENT','type_TRANSFER'], axis =1)
df

,step,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,1,181.00,181.00,0.0,0.00,0.00,1
3,1,181.00,181.00,0.0,21182.00,0.00,1
15,1,229133.94,15325.00,0.0,5083.00,51513.44,0
19,1,215310.30,705.00,0.0,22425.00,0.00,0
24,1,311685.89,10835.00,0.0,6267.00,2719172.89,0
...,...,...,...,...,...,...,...
6351186,699,62372.43,62372.43,0.0,1195387.47,1257759.90,1
6351188,699,162326.52,162326.52,0.0,0.00,0.00,1
6351189,699,162326.52,162326.52,0.0,0.00,162326.52,1
6351190,699,2763398.31,2763398.31,0.0,0.00,0.00,1


In [36]:
df['originError'] = df['oldbalanceOrig']-(df['amount']+df['newbalanceOrig'])
df['destError'] = df['oldbalanceDest']-(df['newbalanceDest']+df['amount'])
df = df.drop(['oldbalanceDest','oldbalanceOrig'], axis = 1)
df

,step,amount,newbalanceOrig,newbalanceDest,isFraud,originError,destError
2,1,181.00,0.0,0.00,1,0.00,-181.00
3,1,181.00,0.0,0.00,1,0.00,21001.00
15,1,229133.94,0.0,51513.44,0,-213808.94,-275564.38
19,1,215310.30,0.0,0.00,0,-214605.30,-192885.30
24,1,311685.89,0.0,2719172.89,0,-300850.89,-3024591.78
...,...,...,...,...,...,...,...
6351186,699,62372.43,0.0,1257759.90,1,0.00,-124744.86
6351188,699,162326.52,0.0,0.00,1,0.00,-162326.52
6351189,699,162326.52,0.0,162326.52,1,0.00,-324653.04
6351190,699,2763398.31,0.0,0.00,1,0.00,-2763398.31


In [37]:
#checking for NA values and null
print('Total of NA values \n')
print(df.isna().sum())
print('\n Total of NA values \n')
print(df.isnull().sum())

Total of NA values 

step              0
amount            0
newbalanceOrig    0
newbalanceDest    0
isFraud           0
originError       0
destError         0
dtype: int64

 Total of NA values 

step              0
amount            0
newbalanceOrig    0
newbalanceDest    0
isFraud           0
originError       0
destError         0
dtype: int64


In [38]:



y = df.isFraud
df = df.drop('isFraud', axis =1)


In [42]:
#df = scaler.fit_transform(df)
#scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
df

array([[-1.71182515, -0.35748584, -0.11431975, -0.43839253,  0.32670344,
         0.33174564],
       [-1.71182515, -0.35748584, -0.11431975, -0.43839253,  0.32670344,
         0.34233534],
       [-1.71182515, -0.09950687, -0.11431975, -0.42737977,  0.08264075,
         0.19407079],
       ...,
       [ 3.26422929, -0.17478395, -0.11431975, -0.40368969,  0.32670344,
         0.16952946],
       [ 3.26422929,  2.75604462, -0.11431975, -0.43839253,  0.32670344,
        -1.04969401],
       [ 3.26422929,  2.75604462, -0.11431975,  0.22496134,  0.32670344,
        -2.43122414]])

In [40]:
import pickle
pickle.dump(scaler, open('scaler.pkl','wb'))

In [ ]:
sc = pickle.load(open('file/path/scaler.pkl','rb'))

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state  = 0)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [10]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression(solver='liblinear', max_iter = 500)

In [11]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [12]:
pred = model.predict(X_test)

In [13]:
print("accuracy",metrics.accuracy_score(y_test,y_pred=pred))
print("precision",metrics.precision_score(y_test,y_pred = pred))
print("f1 score",metrics.f1_score(y_test,y_pred = pred))
print("recall",metrics.recall_score(y_test, y_pred = pred))
print(metrics.classification_report(y_test, y_pred = pred))

accuracy 0.9984073415623097
precision 0.910880829015544
f1 score 0.614900314795383
recall 0.4640971488912355
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    689403
           1       0.91      0.46      0.61      1894

    accuracy                           1.00    691297
   macro avg       0.95      0.73      0.81    691297
weighted avg       1.00      1.00      1.00    691297



In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

In [ ]:
pred = svc.predict(X_test)

In [ ]:
print("accuracy",metrics.accuracy_score(y_test,y_pred=pred))
print("precision",metrics.precision_score(y_test,y_pred = pred))
print("f1 score",metrics.f1_score(y_test,y_pred = pred))
print("recall",metrics.recall_score(y_test, y_pred = pred))
print(metrics.classification_report(y_test, y_pred = pred))

In [14]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 200)

In [15]:
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [16]:
pred = model.predict(X_test)

In [17]:
print("accuracy",metrics.accuracy_score(y_test,y_pred=pred))
print("precision",metrics.precision_score(y_test,y_pred = pred))
print("f1 score",metrics.f1_score(y_test,y_pred = pred))
print("recall",metrics.recall_score(y_test, y_pred = pred))
print(metrics.classification_report(y_test, y_pred = pred))

accuracy 0.9999840878811856
precision 0.9994694960212201
f1 score 0.9970891770309606
recall 0.9947201689545935
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    689403
           1       1.00      0.99      1.00      1894

    accuracy                           1.00    691297
   macro avg       1.00      1.00      1.00    691297
weighted avg       1.00      1.00      1.00    691297



In [18]:
# save the model to disk
filename = 'decision_tree.sav'
pickle.dump(model, open(filename, 'wb'))
 


In [20]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9999840878811856


array([[-1.71182515, -0.35748584, -0.11431975, -0.43839253,  0.32670344,
         0.33174564],
       [-1.71182515, -0.35748584, -0.11431975, -0.43839253,  0.32670344,
         0.34233534],
       [-1.71182515, -0.09950687, -0.11431975, -0.42737977,  0.08264075,
         0.19407079],
       ...,
       [ 3.26422929, -0.17478395, -0.11431975, -0.40368969,  0.32670344,
         0.16952946],
       [ 3.26422929,  2.75604462, -0.11431975, -0.43839253,  0.32670344,
        -1.04969401],
       [ 3.26422929,  2.75604462, -0.11431975,  0.22496134,  0.32670344,
        -2.43122414]])

In [31]:
df={'step':1,'amount':123123,'newbalanceOrig',123123,'newbalanceDest':123123,'originError':1,'destError',123123}

,step,amount,newbalanceOrig,newbalanceDest,isFraud,originError,destError
2,1,181.00,0.0,0.00,1,0.00,-181.00
3,1,181.00,0.0,0.00,1,0.00,21001.00
15,1,229133.94,0.0,51513.44,0,-213808.94,-275564.38
19,1,215310.30,0.0,0.00,0,-214605.30,-192885.30
24,1,311685.89,0.0,2719172.89,0,-300850.89,-3024591.78
...,...,...,...,...,...,...,...
6351186,699,62372.43,0.0,1257759.90,1,0.00,-124744.86
6351188,699,162326.52,0.0,0.00,1,0.00,-162326.52
6351189,699,162326.52,0.0,162326.52,1,0.00,-324653.04
6351190,699,2763398.31,0.0,0.00,1,0.00,-2763398.31
